# MultiAgent with Genie

This notebook demonstrates how to build a multiagent system using [Langgraph](https://blog.langchain.dev/langgraph-multi-agent-workflows/) where [Genie](https://www.databricks.com/product/ai-bi/genie) is one of the agents. The multiagent system can be logged to MLflow as a model and deployed on Databricks with [Agent Framework](https://www.databricks.com/product/machine-learning/retrieval-augmented-generation).  

Install necessary packages

In [0]:
%pip install -U langgraph langchain langchain_experimental databricks-sdk mlflow databricks-langchain
dbutils.library.restartPython()

Enable MLflow Langchain autologging for automatic tracing

In [0]:
import mlflow

mlflow.langchain.autolog()

Create a genie agent with access to a space

In [0]:
from databricks_langchain.genie import GenieAgent

# add your genie space id here
genie_space_id =  "01efa5d3862c1317969a569e3cdfcb1c"
genie_agent = GenieAgent(genie_space_id, "Genie", description="This Genie Agent will have all data about COVID Trials and related articles")

# MultiAgent

Create a StateGraph in Langgraph with a supervisor agent node and other participant agent nodes.

This is inspired by https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/multi_agent/agent_supervisor.ipynb

In [0]:
import os

from databricks_langchain import ChatDatabricks
# add your external model name here
llm = ChatDatabricks(endpoint="srijit_nair_openai")

Create a code agent with access to a python repl

In [0]:
from langchain_experimental.tools import PythonREPLTool
from langgraph.prebuilt import create_react_agent

python_repl_tool = PythonREPLTool()
code_agent = create_react_agent(llm, tools=[python_repl_tool])

Define a supervisor agent

In [0]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel
from typing import Literal

members = ["Genie", "Mathematician"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. If the question has been answered,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members


class routeResponse(BaseModel):
    next: Literal[tuple(options)]


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))


def supervisor_agent(state):
    supervisor_chain = prompt | llm.with_structured_output(routeResponse)
    return supervisor_chain.invoke(state)

Define the multiagent graph structure

In [0]:
import functools
import operator
from typing import Sequence, Annotated
from typing_extensions import TypedDict

from mlflow.langchain.output_parsers import ChatCompletionsOutputParser

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.runnables import RunnableLambda

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [AIMessage(content=result["messages"][-1].content, name=name)]
    }


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


math_node = functools.partial(agent_node, agent=code_agent, name="Mathematician")
genie_node = functools.partial(agent_node, agent=genie_agent, name="Genie")

workflow = StateGraph(AgentState)
workflow.add_node("Genie", genie_node)
workflow.add_node("Mathematician", math_node)
workflow.add_node("supervisor", supervisor_agent)

for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

# parse the output from the graph to get the final message, and then format into ChatCompletions
def get_final_message(resp):
    return resp["messages"][-1]

graph_with_parser = graph | RunnableLambda(get_final_message) | ChatCompletionsOutputParser()

In [0]:
graph_with_parser.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Use only Genie tool and tell me how many COVID trials are in Recruiting status",
            }
        ]
    }
)

set model for mlflow

In [0]:
mlflow.models.set_model(graph_with_parser)